In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
import re

/Users/yumna/workspace/next-word-prediction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '../data/data.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [4]:
print(X.shape)
print(y.shape)


(1628, 39)
(1628, 687)


In [5]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='adam', metrics=['accuracy'])


2026-01-29 10:38:38.569114: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2026-01-29 10:38:38.569246: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-29 10:38:38.569252: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-29 10:38:38.569669: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-29 10:38:38.570028: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X, y, epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
51/51 [==============================] - 1s 21ms/step - loss: 2.0769 - accuracy: 0.5737
Epoch 2/50
51/51 [==============================] - 1s 21ms/step - loss: 2.0214 - accuracy: 0.5934
Epoch 3/50
51/51 [==============================] - 1s 21ms/step - loss: 1.9716 - accuracy: 0.6179
Epoch 4/50
51/51 [==============================] - 1s 21ms/step - loss: 1.9203 - accuracy: 0.6296
Epoch 5/50
51/51 [==============================] - 1s 22ms/step - loss: 1.8699 - accuracy: 0.6327
Epoch 6/50
51/51 [==============================] - 1s 21ms/step - loss: 1.8231 - accuracy: 0.6493
Epoch 7/50
51/51 [==============================] - 1s 21ms/step - loss: 1.7756 - accuracy: 0.6634
Epoch 8/50
51/51 [==============================] - 1s 22ms/step - loss: 1.7286 - accuracy: 0.6671
Epoch 9/50
51/51 [==============================] - 1s 23ms/step - loss: 1.6869 - accuracy: 0.6855
Epoch 10/50
51/51 [==============================] - 1s 23ms/step - loss: 1.6416 - accuracy: 0.6941
Epoch 11/

In [8]:
# Generate next word predictions
seed_text = "I dont like pizza "
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 13ms/step
Next predicted words: I dont like pizza  become a symbol of comfort and techniques creates a beautiful


In [9]:
## Save the model
model.save("next_word_lstm.h5")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

/Users/yumna/workspace/next-word-prediction/.venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
